# Convolutional Neural Network

### Importing the libraries

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
tf.__version__

'2.20.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 21185 images belonging to 2 classes.


### Preprocessing the Test set

In [20]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary',
                                            shuffle = False)

Found 3799 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [11]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [12]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',input_shape=(64, 64, 3)))

C:\Users\VIRA\anaconda3\envs\tf-gpu\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [13]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2))

### Adding a second convolutional layer

In [14]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2))

### Step 3 - Flattening

In [15]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [16]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [17]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [18]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [19]:
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

C:\Users\VIRA\anaconda3\envs\tf-gpu\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
663/663 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.5886 - loss: 0.6677

C:\Users\VIRA\anaconda3\envs\tf-gpu\lib\site-packages\PIL\TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


663/663 ━━━━━━━━━━━━━━━━━━━━ 87s 130ms/step - accuracy: 0.6444 - loss: 0.6271 - val_accuracy: 0.7378 - val_loss: 0.5190
Epoch 2/25
663/663 ━━━━━━━━━━━━━━━━━━━━ 78s 117ms/step - accuracy: 0.7409 - loss: 0.5246 - val_accuracy: 0.7689 - val_loss: 0.4884
Epoch 3/25
663/663 ━━━━━━━━━━━━━━━━━━━━ 72s 109ms/step - accuracy: 0.7682 - loss: 0.4835 - val_accuracy: 0.8134 - val_loss: 0.4201
Epoch 4/25
663/663 ━━━━━━━━━━━━━━━━━━━━ 65s 98ms/step - accuracy: 0.7896 - loss: 0.4536 - val_accuracy: 0.7992 - val_loss: 0.4350
Epoch 5/25
663/663 ━━━━━━━━━━━━━━━━━━━━ 67s 100ms/step - accuracy: 0.7987 - loss: 0.4341 - val_accuracy: 0.7715 - val_loss: 0.4763
Epoch 6/25
663/663 ━━━━━━━━━━━━━━━━━━━━ 68s 103ms/step - accuracy: 0.8040 - loss: 0.4230 - val_accuracy: 0.8105 - val_loss: 0.4219
Epoch 7/25
663/663 ━━━━━━━━━━━━━━━━━━━━ 81s 122ms/step - accuracy: 0.8107 - loss: 0.4090 - val_accuracy: 0.7984 - val_loss: 0.4300
Epoch 8/25
663/663 ━━━━━━━━━━━━━━━━━━━━ 70s 106ms/step - accuracy: 0.8234 - loss: 0.3886 - val_

## Part 4 - Making a single prediction

In [23]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/4.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [24]:
print(prediction)

cat


In [26]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

print("\nStarting detailed evaluation on the test set...")
y_true = test_set.classes
test_set.reset()

steps_to_predict = int(np.ceil(test_set.samples / test_set.batch_size))

y_pred_probs = cnn.predict(test_set, steps=steps_to_predict)
y_pred = (y_pred_probs > 0.5).astype(int)

class_indices = training_set.class_indices
class_names = list(class_indices.keys())

print("\nConfusion Matrix:")
cm = confusion_matrix(y_true, y_pred)
print(cm)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))


Starting detailed evaluation on the test set...
119/119 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step

Confusion Matrix:
[[1140  260]
 [ 308 2091]]

Classification Report:
              precision    recall  f1-score   support

         Cat       0.79      0.81      0.80      1400
         Dog       0.89      0.87      0.88      2399

    accuracy                           0.85      3799
   macro avg       0.84      0.84      0.84      3799
weighted avg       0.85      0.85      0.85      3799

